In [16]:
import requests
requests.packages.urllib3.disable_warnings()
# ignore the http warning
import socket
import re
import json
import time
import pandas as pd

# difine function for url response
def get(url,params=None,headers=None,proxies=None,verify=True,timeout=10,allow_redirects=True,sleep=0,try_num=0):
    try_index=0
    while True:
        try_index+=1
        if try_num!=0 and try_index>=try_num:
            return 
        try:
            response=requests.get(url=url,params=params,headers=headers,proxies=proxies,verify=verify,timeout=timeout,allow_redirects=allow_redirects)
            time.sleep(sleep)
            break
        except requests.exceptions.ProxyError:
            print('get_response——ProxyError')
            continue
        except requests.exceptions.ConnectTimeout:
            print('get_response——ConnectTimeout')
            continue
        except requests.exceptions.ReadTimeout:
            print('get_response——ReadTimeout')
            continue
        except requests.exceptions.ConnectionError:
            print('get_response——ConnectionError')
            continue
        except requests.exceptions.ChunkedEncodingError:
            print('get_response——ChunkedEncodingError')
            continue
        except socket.timeout:
            print('get_response——socket.timeout')
            continue
        except ConnectionResetError:
            print('get_response——ConnectionResetError')
            continue
    return response



In [17]:
# define get the detail function
def get_detail_info(item_url,item_id,all_list):
    response=get(url=item_url,headers=headers,timeout=5,verify=False)
    # print(response.text)
    info_str=re.findall('<script id=hdpApolloPreloadedData type="application/json">(.*?)</script>',response.text,re.S)[0]
    # print(info_str)
    try:
        info_dict=json.loads(json.loads(info_str)['apiCache'])['NotForSaleShopperPlatformFullRenderQuery{"zpid":'+str(item_id)+',"contactFormRenderParameter":{"zpid":'+str(item_id)+',"platform":"desktop","isDoubleScroll":true}}']['property']
    except KeyError:
        return
    bedrooms=info_dict.get('bedrooms','')
    bathrooms=info_dict.get('bathrooms','')
    livingAreaValue=info_dict.get('livingAreaValue','')
    price=info_dict.get('price','')
    dateSoldString=info_dict.get('dateSoldString','')
    zestimate=info_dict.get('zestimate','')

    fact_dict={}
    atAGlanceFacts=info_dict['resoFacts']['atAGlanceFacts']
    for fact in atAGlanceFacts:
        key=fact['factLabel']
        value=fact['factValue']
        fact_dict[key]=value
    # toolkit.pp(fact_dict)
    Type=fact_dict.get('Type','')
    Cooling=fact_dict.get('Cooling','')
    Built=fact_dict.get('Year Built','')
    Parking=fact_dict.get('Parking','')
    Heating=fact_dict.get('Heating','')
    Lot=fact_dict.get('Lot','')
    # all the variables    
    info=[item_url,bedrooms,bathrooms,livingAreaValue,price,dateSoldString,zestimate,Type,Cooling,Built,Parking,Heating,Lot]
    #print(info)
    #if need to check the details uncomment the print
    all_list.append(info)

In [18]:
# my device is mac
headers={
    'cookie': 'zguid=23|%24ce81dac8-d941-41cd-a673-237494d60771; zgsession=1|3daadb36-07f1-40ff-996e-ac262a7168c0; _ga=GA1.2.1299414427.1639355151; zjs_user_id=null; zjs_anonymous_id=%22ce81dac8-d941-41cd-a673-237494d60771%22; _pxvid=3a4f8c84-5bab-11ec-8d40-567153697661; _gcl_au=1.1.1162599933.1639355154; KruxPixel=true; DoubleClickSession=true; __pdst=2f046e2aafe14a959e179e5c60f9b2a9; _fbp=fb.1.1639355154288.1552151640; _pin_unauth=dWlkPVl6bGtOakl4T0dFdFptUXhOUzAwT1RCbExUbGlObU10T0RCaU5qQTJNVEkxWVRVeQ; __gads=ID=8c2e57d206a3d5c9:T=1639355155:S=ALNI_MY6rE6QR_YGTWNt9e1QKO8MVA9JCg; KruxAddition=true; _gid=GA1.2.323551660.1639452437; g_state={"i_p":1639459642544,"i_l":1}; utag_main=v_id:017db12e7f400009731522ba12ab0507803ee0700088e$_sn:2$_se:1$_ss:1$_st:1639454238736$dc_visit:2$ses_id:1639452438736%3Bexp-session$_pn:1%3Bexp-session$dcsyncran:1%3Bexp-session$tdsyncran:1%3Bexp-session$dc_event:1%3Bexp-session$dc_region:us-east-1%3Bexp-session$ttd_uuid:a41cb28e-78fd-4d9f-95de-32af263fd779%3Bexp-session; _gac_UA-21174015-56=1.1639454108.CjwKCAiA-9uNBhBTEiwAN3IlNE5NMRvWCONluxxth4Cr6hn1eNoQwylF5Stmdl8a68nl36pbv_jA8hoCtiMQAvD_BwE; _gcl_aw=GCL.1639454109.CjwKCAiA-9uNBhBTEiwAN3IlNE5NMRvWCONluxxth4Cr6hn1eNoQwylF5Stmdl8a68nl36pbv_jA8hoCtiMQAvD_BwE; JSESSIONID=8A95F7E3EFF94535F714D1D9179BABBA; _uetsid=bddab9805c8d11ecaef10b27730c9194; _uetvid=3bcab1605bab11eca91317129fdd127e; _px3=919c1b8118d9097dd20c73417d9d57324e99e4e0737d155b0d7c862770b900b7:qRHyOAkawL7Pk6romOY0CyYoTlTjkspdE83TqCuylUgodNH9Rbcz45sv7oFbRxOT4IjaFWS9dWDoHNrgVKXhlQ==:1000:50J/UFVg+1kn2G9V71aHd16/qo3TL+qnYA2tQPXHLX/i5QI92mwSkWhugTki3Ormt0rOlLbvS/gKIowQ4bTSPxWR9aWRdXh8airST9kl8oE50xf72IlIyoHYEERApB1jFs5+JSFwy/H0amf5Fb3nhBTPyrWQa9AYKl8UzqboJvtc1ie9YKELSYrQ+WbyGPSmaC3/5TpQ0lLBDBiDdZN2SQ==; _gat=1; AWSALB=C+nFhEuBBVOOuFMFBunQDwSQ7w2FyG7pML0R+JOMMHsJ91fYoZYvkNYGAeOE5q1nU+gtuJCy0+nO/QZy1aJEAeAKtSIzpZYk/OATRN66lTNKIiN7cJeVeipN7kJX; AWSALBCORS=C+nFhEuBBVOOuFMFBunQDwSQ7w2FyG7pML0R+JOMMHsJ91fYoZYvkNYGAeOE5q1nU+gtuJCy0+nO/QZy1aJEAeAKtSIzpZYk/OATRN66lTNKIiN7cJeVeipN7kJX; search=6|1642083956979%7Crect%3D44.423188887203736%252C-66.77469659375%252C37.078408640436265%252C-79.12332940625%26rid%3D784024%26disp%3Dmap%26mdm%3Dauto%26p%3D1%26sort%3Ddays%26z%3D1%26days%3D90%26type%3Dhouse%26fs%3D0%26fr%3D0%26mmm%3D0%26rs%3D1%26ah%3D0%26singlestory%3D0%26housing-connector%3D0%26abo%3D0%26garage%3D0%26pool%3D0%26ac%3D0%26waterfront%3D0%26finished%3D0%26unfinished%3D0%26cityview%3D0%26mountainview%3D0%26parkview%3D0%26waterview%3D0%26hoadata%3D1%26zillow-owned%3D0%263dhome%3D0%26featuredMultiFamilyBuilding%3D0%09%0957124%09%09%09%09%09%09',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'
}

In [19]:
# initial the dataset to store the crawing details
all_list=[]

In [20]:
# zillow has unti-crawing system, and each link can only cral 20pages
for page_num in range(1,20):
    api='https://www.zillow.com/search/GetSearchPageState.htm'
    params={
        'searchQueryState': '{"pagination":{"currentPage":'+str(page_num)+'},"usersSearchTerm":"Long Island, NY","mapBounds":{"west":-76.1625144765555,"east":-69.7355125234305,"south":40.15074949590003,"north":41.546824028651706},"mapZoom":8,"regionSelection":[{"regionId":784024,"regionType":31}],"isMapVisible":true,"filterState":{"doz":{"value":"90"},"isCondo":{"value":false},"isForSaleForeclosure":{"value":false},"isApartment":{"value":false},"isAllHomes":{"value":true},"isAuction":{"value":false},"isNewConstruction":{"value":false},"isRecentlySold":{"value":true},"isLotLand":{"value":false},"isForSaleByOwner":{"value":false},"isComingSoon":{"value":false},"isApartmentOrCondo":{"value":false},"isForSaleByAgent":{"value":false}},"isListVisible":true}',
        'wants': '{"cat1":["listResults","mapResults"]}',
        'requestId': '3'
    }
    response=get(url=api,params=params,headers=headers,timeout=5,verify=False)
    response_json=response.json()
    # toolkit.pp(json)

    item_list=response_json['cat1']['searchResults']['listResults']
    print(len(item_list))
    for item in item_list:
        # toolkit.pp(item)
        item_id=item['id']
        item_url=item['detailUrl']
        # print(item_id)
        # print(item_url)
        get_detail_info(item_url=item_url,item_id=item_id,all_list=all_list)
        time.sleep(1)
        # break
    # break

40
['https://www.zillow.com/homedetails/1515-New-Hyde-Park-Dr-New-Hyde-Park-NY-11040/31100772_zpid/', 5, 2, 1837, 830000, '2021-12-14', 838600, 'Single Family Residence', 'Central Air', '1946', '1 Garage space', 'Natural Gas, Baseboard', '5,000 sqft']
['https://www.zillow.com/homedetails/151-Sutton-Dr-Plainview-NY-11803/31136232_zpid/', 3, 2, 1469, 675000, '2021-12-14', 646800, 'Single Family Residence', 'Central Air', '1952', '1 Attached Garage space', 'Oil, Forced Air', '8,000 sqft']
['https://www.zillow.com/homedetails/2365-1st-St-East-Meadow-NY-11554/31327077_zpid/', 4, 2, None, 627500, '2021-12-14', 667300, 'Single Family Residence', None, '1948', 'Private', None, '6,000 sqft']
['https://www.zillow.com/homedetails/25-Hurtin-St-Port-Jefferson-Station-NY-11776/59477536_zpid/', 4, 3, 2340, 555000, '2021-12-13', 567700, 'Single Family Residence', 'Central Air', '1970', '2 Attached Garage spaces', 'Oil, Baseboard', '9,148 sqft']
['https://www.zillow.com/homedetails/30-Friendly-Ln-Jeric

['https://www.zillow.com/homedetails/105-Lawrence-Hill-Rd-Cold-Spring-Harbor-NY-11724/59560630_zpid/', 4, 4, None, 2144000, '2021-12-13', 2049400, 'Single Family Residence', 'Central Air', '1969', '2 Attached Garage spaces', 'Natural Gas, Hot Water', None]
['https://www.zillow.com/homedetails/13-Manhattan-Blvd-Islip-Terrace-NY-11752/59631574_zpid/', 3, 2, None, 515000, '2021-12-13', 486200, 'Single Family Residence', 'ENERGY STAR Qualified Equipment, Window Unit(s)', '1910', '2 Garage spaces', 'Natural Gas, Baseboard, Hot Water, ENERGY STAR Qualified Equipment', '5,663 sqft']
['https://www.zillow.com/homedetails/356-Keller-Ave-Elmont-NY-11003/31180383_zpid/', 5, 3, 2078, 789000, '2021-12-13', 773500, 'Single Family Residence', 'Window Unit(s)', '1975', '1 Attached Garage space', 'Oil, Baseboard', '4,000 sqft']
40
['https://www.zillow.com/homedetails/175-Evergreen-Ave-Smithtown-NY-11787/32704942_zpid/', 3, 2, None, 615000, '2021-12-13', 588800, 'Single Family Residence', 'Wall Unit(s), 

['https://www.zillow.com/homedetails/25-Clark-Ave-Lynbrook-NY-11563/31232439_zpid/', 4, 3, None, 730000, '2021-12-13', 748800, 'Single Family Residence', 'Central Air', '1923', '1 Garage space', 'Natural Gas, Baseboard, Hot Water', '5,000 sqft']
['https://www.zillow.com/homedetails/8210-Pettit-Ave-Flushing-NY-11373/31964643_zpid/', 6, 3, None, 1500000, '2021-12-13', 1231800, 'Multi Family', 'Window Unit(s)', '1925', '2 Attached Garage spaces', 'Oil, Hot Water', '']
['https://www.zillow.com/homedetails/105-Bay-50th-St-Brooklyn-NY-11214/2107111349_zpid/', 12, 6, 3900, 700000, '2021-12-13', 1112900, 'MultiFamily', None, '1931', '0 spaces', None, '']
['https://www.zillow.com/homedetails/107-Bay-50th-St-Brooklyn-NY-11214/30731771_zpid/', 4, 12, 3240, 550000, '2021-12-13', None, 'MultiFamily', None, '1935', '0 spaces', None, '']
['https://www.zillow.com/homedetails/111-Bay-50th-St-Brooklyn-NY-11214/30731770_zpid/', 4, 2, 3430, 700000, '2021-12-13', None, 'MultiFamily', None, '1945', '0 space

['https://www.zillow.com/homedetails/11-View-Dr-Miller-Place-NY-11764/59316161_zpid/', 5, 4, 4400, 1250000, '2021-12-10', 550800, 'Single Family Residence', 'Central Air', '1986', '2 Attached Garage spaces', 'Oil, Forced Air', '1.79 Acres']
['https://www.zillow.com/homedetails/111-Barnyard-Ln-Levittown-NY-11756/31334128_zpid/', 3, 1, 1250, 552000, '2021-12-10', 562100, 'Single Family Residence', 'Central Air', '1948', 'Private, Driveway', 'Other, Forced Air, Heat Pump, Hot Water, ENERGY STAR Qualified Equipment', '6,000 sqft']
['https://www.zillow.com/homedetails/15026-78th-Rd-Flushing-NY-11367/32069886_zpid/', 3, 2, 1332, 750000, '2021-12-10', 791200, 'Single Family Residence', 'Wall Unit(s), Window Unit(s)', '1945', '1 Garage space', 'Oil, Hot Water', '1,800 sqft']
['https://www.zillow.com/homedetails/23-Cedar-Heights-Dr-Ridge-NY-11961/59548886_zpid/', 3, 3, None, 500000, '2021-12-10', 505500, 'Single Family Residence', 'Central Air', '1985', '1 Attached Garage space', 'Oil, Baseboar

['https://www.zillow.com/homedetails/124-Norman-St-New-Hyde-Park-NY-11040/31109978_zpid/', 4, 2, 1152, 737000, '2021-12-10', 753700, 'SingleFamily', 'Central', '1943', 'Off-street, On-street', 'Other', '5,000 sqft']
['https://www.zillow.com/homedetails/69-Florence-Ave-Hempstead-NY-11550/31218418_zpid/', 3, 2, None, 460000, '2021-12-10', 463800, 'Single Family Residence', 'Wall Unit(s)', '1926', '1 Garage space', 'Oil, Baseboard', '4,200 sqft']
['https://www.zillow.com/homedetails/777-Janos-Ln-West-Hempstead-NY-11552/31213351_zpid/', 3, 2, None, 510000, '2021-12-10', 517500, 'Single Family Residence', 'None', '1952', '1 Attached Garage space', 'Oil, Other', '6,014 sqft']
['https://www.zillow.com/homedetails/2837-Robert-Ln-Bellmore-NY-11710/31391886_zpid/', 4, 3, None, 641200, '2021-12-10', 652700, 'Single Family Residence', 'Wall Unit(s)', '1970', '1 Attached Garage space', 'Natural Gas, Baseboard, Hot Water', '6,348 sqft']
['https://www.zillow.com/homedetails/10226-Nicolls-Ave-Flushing

...

['https://www.zillow.com/homedetails/8117-17th-Ave-Brooklyn-NY-11214/30710335_zpid/', None, None, 2820, 1230000, '2021-12-08', 1290200, 'MultiFamily', None, '1920', '0 spaces', None, '']
['https://www.zillow.com/homedetails/11121-Inwood-St-Jamaica-NY-11435/32180446_zpid/', 3, 2, 1248, 590000, '2021-12-08', 596000, 'Single Family Residence', 'Window Unit(s)', '1930', 'Driveway, Private', 'Natural Gas, Hot Water', None]
40
['https://www.zillow.com/homedetails/6256-70th-St-Flushing-NY-11379/31994750_zpid/', None, None, 3245, 1318000, '2021-12-08', 1322400, 'MultiFamily', None, '1965', 'Garage, Garage - Attached', None, '']
['https://www.zillow.com/homedetails/2027-66th-St-Brooklyn-NY-11204/30682108_zpid/', None, None, 1580, 1320000, '2021-12-08', 1195200, 'MultiFamily', None, '1915', '0 spaces', None, '']
['https://www.zillow.com/homedetails/464-43rd-St-Brooklyn-NY-11232/30575073_zpid/', 6, 3, 3072, 1600000, '2021-12-08', 1533800, 'MultiFamily', None, '1901', '0 spaces', None, '']
['https

['https://www.zillow.com/homedetails/69-Churchill-Ln-Smithtown-NY-11787/32698136_zpid/', 4, 4, None, 752000, '2021-12-07', 739700, 'Single Family Residence', 'Central Air', '1985', '2 Attached Garage spaces', 'Natural Gas, Hot Water', '0.64 Acres']
['https://www.zillow.com/homedetails/1035-Bay-32nd-St-Far-Rockaway-NY-11691/55732756_zpid/', 6, 5, 2392, 740000, '2021-12-07', 753100, 'Multi Family', 'Wall Unit(s)', '1998', 'Private, Driveway, No Garage', 'Natural Gas, Steam', '']
['https://www.zillow.com/homedetails/61-7th-St-Valley-Stream-NY-11581/31251129_zpid/', 4, 2, 2000, 810000, '2021-12-07', 846500, 'Multi Family', 'Ductless', '1917', '4 Parking spaces', 'Natural Gas, Baseboard', '']
['https://www.zillow.com/homedetails/207-Syracuse-Ave-Medford-NY-11763/59520293_zpid/', 3, 2, None, 385000, '2021-12-07', 402700, 'Single Family Residence', 'Zoned', '1990', 'Private, Driveway', 'Oil, Baseboard, Hot Water', '5,227 sqft']
40
['https://www.zillow.com/homedetails/53-Annuskemunnica-Rd-Baby

['https://www.zillow.com/homedetails/1438-Pine-Dr-Bay-Shore-NY-11706/59636752_zpid/', 4, 2, None, 525000, '2021-12-07', 470200, 'Single Family Residence', None, '1977', 'Private, Driveway, No Garage, None', 'Natural Gas, See Remarks', '9,583 sqft']
['https://www.zillow.com/homedetails/18-Gull-Dr-Hauppauge-NY-11788/32708817_zpid/', 3, 2, None, 575000, '2021-12-07', 567300, 'Single Family Residence', 'Wall Unit(s)', '1965', '1 Attached Garage space', 'Natural Gas, Forced Air', '0.26 Acres']
['https://www.zillow.com/homedetails/1661-Lydia-Ave-Elmont-NY-11003/31181960_zpid/', 4, 3, 1826, 671000, '2021-12-07', 623100, 'Single Family Residence', 'Wall Unit(s), Window Unit(s)', '1951', '1 Garage space', 'Natural Gas, Steam', '6,816 sqft']
['https://www.zillow.com/homedetails/197-Clinton-Ave-E-Patchogue-NY-11772/2067329835_zpid/', 3, 2, None, 380000, '2021-12-07', None, 'Single Family Residence', 'Window Unit(s)', '1986', '1 Attached Garage space', 'Oil, Baseboard', '6,970 sqft']
40
['https://

['https://www.zillow.com/homedetails/16008-Grand-Central-Pkwy-Jamaica-NY-11432/32073465_zpid/', 4, 2, None, 820000, '2021-12-07', 821400, 'Single Family Residence', 'None', '1930', '1 Garage space', 'Natural Gas, Steam', '2,758 sqft']
['https://www.zillow.com/homedetails/10404-29th-Ave-Flushing-NY-11369/31968738_zpid/', 7, 2, None, 1400000, '2021-12-07', 1490000, 'Multi Family', None, '1920', '6 Attached Garage spaces', 'Natural Gas, See Remarks', '']
['https://www.zillow.com/homedetails/18503-Nashville-Blvd-Jamaica-NY-11413/32195513_zpid/', None, None, 1046, 400000, '2021-12-07', 544200, 'SingleFamily', None, '1925', 'Garage, Garage - Attached', None, '4,000 sqft']
['https://www.zillow.com/homedetails/10929-110th-St-Jamaica-NY-11420/32169356_zpid/', None, None, 1436, 10000, '2021-12-07', 757000, 'MultiFamily', None, '1935', 'Garage, Garage - Attached', None, '']
['https://www.zillow.com/homedetails/3764-63rd-St-Flushing-NY-11377/31956236_zpid/', None, None, 2320, 10, '2021-12-07', 111

['https://www.zillow.com/homedetails/5-Magnolia-Dr-Massapequa-NY-11758/31353315_zpid/', 4, 1, 2100, 565000, '2021-12-07', 561700, 'Single Family Residence', 'Wall Unit(s)', '1954', '1 Garage space', 'Natural Gas, Baseboard', '8,200 sqft']
['https://www.zillow.com/homedetails/163-Alice-Ave-Oceanside-NY-11572/31408784_zpid/', 3, 1, None, 509000, '2021-12-07', 503300, 'Single Family Residence', 'Ductless', '1935', 'Private, Driveway', 'Natural Gas, Baseboard, ENERGY STAR Qualified Equipment', '5,000 sqft']
['https://www.zillow.com/homedetails/3-Peacock-Pond-Rd-Glen-Cove-NY-11542/31157248_zpid/', 3, 4, 3797, 2025000, '2021-12-07', 1700700, 'Single Family Residence', 'Central Air', '1986', '2 Attached Garage spaces', 'Natural Gas, Baseboard', '1.13 Acres']
['https://www.zillow.com/homedetails/240-Belvedere-Dr-Massapequa-Park-NY-11762/31393749_zpid/', 4, 2, 1591, 525000, '2021-12-07', 538700, 'Single Family Residence', 'Window Unit(s)', '1950', '1 Attached Garage space', 'Oil, Baseboard', '6

In [21]:
# save the the data to excel
df=pd.DataFrame(data=all_list)
df.to_excel('zillow.xlsx',index=False)